In [1]:
import sys, os

root_dir = os.path.join(os.getcwd(), '..')
sys.path.append(root_dir)

from src.adaptation import Adaptation

C:\Users\Eric\miniconda3\envs\mpro\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
a = Adaptation()
a.available_operations

[src.adaptation.operations.start_end_on_ct_operation.StartEncOnCTOperation,
 src.adaptation.operations.transpose_notes_operation.TransposeNotesOperation,
 src.adaptation.operations.transpose_sequence_operation.TransposeSequenceOperation]